In [1]:
#Codigo para clasificar mensajes recibidos + matchear
#Ultima actualizacion: 04/Ago/2022

# Inputs: 
#       (A) sent_mails_conf_envio_[FECHA] (del codigo anterior), que tiene las respuestas que se enviaron efectivamente y 
#       (B) base de mails con respuestas de inmobiliarias (puede haber de properati, no es problema).
# Outputs: 
#       (A) respuestas_inmobiliarias_[FECHA], que agrega TODAS las publicaciones y respuestas, matcheadas o no (despues se puede filtrar por lo que queramos), y 
#       (B) resultados_matching_rtas_[FECHA] que muestra, para cada mail, la cantidad de respuestas matcheadas y los % de match.


In [5]:
# Importar paquetes necesarios para la clasificacion

import pandas as pd
import numpy as np
from datetime import datetime
import matching_v1 as matching
import tools

pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'

#Establecer rutas de entrada y salida para las bases (si se cambian, comentar las que no se usen)

# Ronda y fecha de la ronda
ronda = '1'
fecha = '2022_04_19'
ronda2 = '2'
fecha2 = '2022_05_04'

fecha_descarga = '2022_05_19'

#Rutas Nico
path_dropbox = r"C:\Users\Nico\Dropbox"

#Rutas Juli
# path_data = r"C:\Users\Nico\Dropbox\BID-LGBTQ+\campo"

#Rutas Luis
# path_data = r"C:\Users\Nico\Dropbox\BID-LGBTQ+\campo"

#Rutas generadas:
path_data = path_dropbox + r"\BID-LGBTQ+\campo\data"
path_datain = path_data + r"\mails_matches"
path_dataout = path_data + fr"\mails_matches\matched\respuestas_inmobiliarias\round{ronda}"
path_deleted = path_data + r"\mails_matches\deleted"

In [6]:
#Importar base de mails recibidos (todos)
rtas_full = pd.read_excel(path_datain + fr"\to_match\mails_total_2022_05_19.xlsx") # 2022_05_19 Ultima descarga de rtas de gmail

#Importar base de mails con confirmación de envios
enviados = pd.read_excel(path_datain + fr"\matched\confirmaciones_de_envio\sent_mails_conf_envio_round_{ronda}_2022_08_04.xlsx")
enviados = enviados.drop(columns=['Unnamed: 0.1','_merge'])
enviados = enviados[-enviados.body.str.contains("miberniel")] # Elimino las observaciones que la propiedad de Inés

In [7]:
#Importo bases de seleccion (tienen la variable nombre, n_adds_abv_mun y algunas otras)
files = tools.crea_lista_de_archivos(path_data + fr"\scrape\selection\round1", "dta")
files = [f for f in files if 'scrape_select' in f]

df_concat = []
for file in files:
    df_temp = pd.read_stata(file)
    df_concat += [df_temp]
    
select_full = pd.concat(df_concat)
select_full = select_full.reset_index(drop=True)
select_full = select_full.drop_duplicates(subset=['link','id'], keep='first')

#Formateo variable 'cocheras'
enviados['cocheras'] = pd.to_numeric(enviados.cocheras, errors='coerce')
select_full['cocheras'] = pd.to_numeric(select_full.cocheras, errors='coerce')

#Agrego variable "nombre"
enviados = enviados.merge(select_full, on = ['link','id'], how  = 'outer', indicator=True)
print(enviados._merge.value_counts())

# Como left_only == 0 y both=3633 (observaciones que tenía en confirmadas en enviados), está todo ok! (nota: las right_only son observaciones de la ronda 2!!!)
# Hago el inner join asi vuelo las que no son de esta ronda (dropeo las que no son both)
enviados = enviados[enviados['_merge']=='both']
enviados = tools.normaliza_columnas_post_merge(enviados, suffixes=('_x','_y'))

both          3633
right_only    3595
left_only        0
Name: _merge, dtype: int64
Hay valores diferentes en las columans inmobiliaria_y y inmobiliaria_x


,inmobiliaria_x,inmobiliaria_y
82,duchini propiedades,palchevich propiedades
136,conin propiedades,botto nunez propiedades


Hay valores diferentes en las columans fecha_y y fecha_x


,fecha_x,fecha_y
82,2022-02-02,2022-04-11
136,2022-03-08,2022-03-16


Hay valores diferentes en las columans p_m2_above_mdn_y y p_m2_above_mdn_x


,p_m2_above_mdn_x,p_m2_above_mdn_y
136,1.0,0.0


Hay valores diferentes en las columans n_adds_y y n_adds_x


,n_adds_x,n_adds_y
82,8.0,3.0


Hay valores diferentes en las columans common_name_y y common_name_x


,common_name_x,common_name_y
82,2.0,1.0


In [8]:
enviados = tools.normaliza_columnas_post_merge(enviados, suffixes=('_x','_y'))

In [9]:
enviados.loc[[82,136],:]

,link,id,treatment,ses,male_name,mail,male_couple,female_couple,script,script_string,script_string_2,script_string_3,randomization_seed,index_rtas,Unnamed: 0,subject,from,to,body,date,cc,thread_id,mimeType,conf_envio,date_clean,index_scrap,match_string,match2_string,pais,Unnamed01,Unnamed011,_merge,match_confiab,match_step,nombre,inmobiliaria,fecha,municipio,ciudad,descripcion,nombre_2,precio,moneda,caracteristicas,ambientes,habitaciones,banos,m2_totales,m2_cubiertos,cocheras,profesional,vivienda,precio_local,p_m2,p_m2_above_mdn,n_adds,n_adds_above_mdn,n_adds_muni,n_adds_muni_above_mdn,m2_above_mdn,common_name,capital,random
82,properati.com.ar/detalle/49c41_alquiler_depart...,dch dch 325,homo,neutral,Pablo,fh.pablo.gomez@gmail.com,Juan,NaN,1.0,"Buenos días, mi nombre es Pablo, junto a mi pa...",Pablo y Juan.,Código dch dch 325,20.0,0.0,772.0,Hiciste una consulta por la propiedad Loria Al...,Properati <contacts@mailing.properati.com.ar>,fh.pablo.gomez@gmail.com,[<p>-- SOLICITUD DE ENTREVISTA --\r\n\r\n*DATO...,"Mon, 25 Apr 2022 16:25:21 +0000 (UTC)",NaN,180618ab8706bb12,multipart/alternative,True,2022-04-25,89.0,"[Match(start=1426, end=1441, dist=0, matched='...",NaN,arg,NaN,NaN,both,100.0,1.0,departamento en alquiler en uruguay 900. entre...,duchini propiedades,2022-02-02,caba,buenos aires ampliado,"departamento 2 ambientes , contrafrente con p...",departamento en alquiler en loria al 100,28000.0,$,"['2ambientes', '1bano']",2.0,NaN,1.0,NaN,NaN,NaN,0.0,0.0,28000.0,NaN,NaN,8.0,1.0,3.0,1.0,NaN,2.0,1.0,0.6544
136,properati.com.ar/detalle/4cz6e_alquiler_depart...,617935,homo,neutral,Manuel,jcmanuel.rodriguez@gmail.com,Luis,NaN,1.0,"Buenos días, mi nombre es Manuel, junto a mi p...",Manuel y Luis.,Código 617935,20.0,1.0,2488.0,Hiciste una consulta por la propiedad Esmerald...,Properati <contacts@mailing.properati.com.ar>,jcmanuel.rodriguez@gmail.com,[<p>-- SOLICITUD DE ENTREVISTA --\r\n\r\n*DATO...,"Mon, 25 Apr 2022 16:24:27 +0000 (UTC)",NaN,1806189e2fa979f7,multipart/alternative,True,2022-04-25,85.0,"[Match(start=1364, end=1376, dist=0, matched='...",NaN,arg,NaN,NaN,both,100.0,1.0,departamento en alquiler en esmeralda 900,conin propiedades,2022-03-08,caba,buenos aires ampliado,departamento en alquiler ubicado en esmeralda ...,departamento en alquiler en esmeralda 900,49000.0,$,"['2habitaciones', '1bano', '42 m²totales']",NaN,2.0,1.0,42.0,NaN,NaN,0.0,0.0,49000.0,1166.6666,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.8712


In [10]:
#### Elimino observaciones que se corresponden a mails de prueba o de Google

# Dropeo observaciones que tienen Google en el mail / user
prev = len(rtas_full)
rtas_full_out = rtas_full[rtas_full['from'].str.contains("Google", na=False) == True]
rtas_full =     rtas_full[rtas_full['from'].str.contains("Google", na=False) == False]
print("Se dropearon", prev - len(rtas_full), "observaciones.")

# Dropeo observaciones de prueba de Juli
prev = len(rtas_full) 
to_concat = rtas_full[rtas_full['from'].str.contains("Julian test", na=False) == True]
rtas_full = rtas_full[rtas_full['from'].str.contains("Julian test", na=False) == False]
rtas_full_out = pd.concat([rtas_full_out,rtas_full[rtas_full['from'].str.contains("Julian test", na=False) == True]])

print("Se dropearon", prev - len(rtas_full), "observaciones.")

# Dropeo observaciones de prueba
prev = len(rtas_full) 
to_concat = rtas_full[rtas_full['subject'].str.contains("prueba", na=False) == True]
rtas_full = rtas_full[rtas_full['subject'].str.contains("prueba", na=False) == False]
rtas_full_out = pd.concat([rtas_full_out,to_concat])
print("Se dropearon", prev - len(rtas_full), "observaciones.")

# Base de correos desde los que enviamos
mails_df = pd.read_excel(path_dropbox + r"\BID-LGBTQ+\campo\0_instrucciones\2-Emails para Intervención.xlsx")
mails_df['Usuario_clean'] = mails_df.Usuario.apply(matching.clean_mail)
users = mails_df['Usuario_clean'].to_list()

# Dropeo observaciones de prueba (from == to, ¡nos los autoenviamos!)
prev = len(rtas_full) 
for user in users:
    to_concat = rtas_full[rtas_full['from'].apply(matching.clean_mail).str.contains(user)==True ]
    rtas_full = rtas_full[rtas_full['from'].apply(matching.clean_mail).str.contains(user)==False]
    rtas_full_out = pd.concat([rtas_full_out,to_concat])
print("Se dropearon", prev - len(rtas_full), "observaciones.")

Se dropearon 423 observaciones.
Se dropearon 221 observaciones.
Se dropearon 146 observaciones.
Se dropearon 656 observaciones.


In [11]:
##### Identifica mails que vienen de properati 'contacts@mailing.properati.com'

# Filtro nulls
before = len(rtas_full)
rtas_full = rtas_full.dropna(subset=['from','body'])
after = len(rtas_full)
print("Se dropearon", before-after, "observaciones.")

# Creo binaria con True para los mails de properati
mails_properati = ['contacts@mailing.properati.com']

rtas_full['conf_envio'] = rtas_full['from'].apply(matching.match_string_con_lista, lista_matchs=mails_properati)
print("Match confirmaciones realizado.\nResultados:")

# Reporta los resultados
rtas_full['conf_envio'].value_counts()

Se dropearon 20 observaciones.
Match confirmaciones realizado.
Resultados:


True     7281
False    4931
Name: conf_envio, dtype: int64

In [12]:
enviados = enviados[enviados.mail.notnull()]

In [13]:
##### Limpio las fechas
rtas_full['date_clean'] = rtas_full['date'].str.replace(".+,\s","",regex=True)
rtas_full['date_clean'] = rtas_full['date_clean'].str.replace("\-|\+|(0\d00)(.*)","",regex=True)
rtas_full['date_clean'] = rtas_full['date_clean'].str.replace("GMT","")
rtas_full['date_clean'] = rtas_full['date_clean'].str.lstrip(' ').str.rstrip(' ')
rtas_full['date_clean'] = rtas_full['date_clean'].str.zfill(20)
rtas_full['date_clean'] = pd.to_datetime(rtas_full['date_clean'], format="%d %b %Y %H:%M:%S")
rtas_full['date_clean'] = rtas_full['date_clean'].dt.date  # Solo me quedo dia-mes-año

In [14]:
##### Matcheo respuestas (ya mergeadas con @properati) con scraps

# lista_paises=[]
fin_ronda_1 = pd.to_datetime("2022-05-03").date()
rtas_inmob = rtas_full.loc[(rtas_full['date_clean']<fin_ronda_1) & (rtas_full['conf_envio']==False)]

today = datetime.now()
today = today.strftime("%Y_%m_%d")
nombre = f'mails_total_matched_round_{ronda}_{today}'

rtas_inmob['index_scrap'] = np.nan
rtas_inmob['match_confiab'] = np.nan
rtas_inmob['match_step'] = np.nan
rtas_inmob['match_string'] = np.nan
rtas_inmob['match2_string'] = np.nan
enviados['index_rtas'] = np.nan
enviados['match_confiab'] = np.nan
enviados['match_step'] = np.nan

matching.full_proceso(path=path_dataout, rtas_full=rtas_inmob, scrap_full=enviados)


###########################################################
Cantidad de Respuestas: 2402 
###########################################################
Limpio nombre de inmobiliarias, precios y calles:
palabras eliminadas: ['en', 'alquiler', 'departamento', 'al', '', 'de', 'y', 'entre', 'arriendo', 'apartamento', 'av.']
palabras eliminadas: []
palabras eliminadas: ['propiedades', 'inmobiliaria', 'inmobiliarios', '', 'bienes', 'negocios', 'raices', 'inmobiliario']
Despues de la limpieza... ¿hay repetidas?
False    2732
True      901
Name: inmobiliaria_c, dtype: int64


100%|██████████| 192/192 [09:44<00:00,  3.05s/it]


C:\Users\Nico\Dropbox\BID-LGBTQ+\campo\data\mails_matches\matched\respuestas_inmobiliarias\round1\resultados_matching_rtas_2022_08_11.xlsx creado
C:\Users\Nico\Dropbox\BID-LGBTQ+\campo\data\mails_matches\matched\respuestas_inmobiliarias\round1\respuestas_inmobiliarias_2022_08_11_matched.xlsx creado
end of script


,Matchs,Porcentaje,Total
rodriguezmanuel.w@gmail.com,9,0.409091,22
rodriguezmanuelw@gmail.com,11,0.687500,16
gomez.pablo.u@gmail.com,17,0.653846,26
fh.pablo.gomez@gmail.com,21,0.512195,41
gomez.pablou@gmail.com,10,0.454545,22
...,...,...,...
bflores.danielj@gmail.com,4,0.500000,8
bdominguezalejandroc@gmail.com,3,0.750000,4
bflores.daniel.j@gmail.com,3,0.600000,5
bdominguez.alejandro.c@gmail.com,1,0.250000,4


In [15]:
# # Importo todos los outs y mergeo en una única

# rtas_ARG = pd.read_excel(path_datain + fr'\mails_ARG_round_1_{fecha_rtas}.xlsx')
# rtas_COL = pd.read_excel(path_datain + fr'\mails_COL_round_1_{fecha_rtas}.xlsx')
# rtas_ECU = pd.read_excel(path_datain + fr'\mails_ECU_round_1_{fecha_rtas}.xlsx')
# rtas_PER = pd.read_excel(path_datain + fr'\mails_PER_round_1_{fecha_rtas}.xlsx')

# rtas_full = pd.concat([rtas_ARG,rtas_COL,rtas_ECU,rtas_PER])

# #Importar base de scraps 
# fecha_scrap = '2020_04_11'
# scrap_ARG = pd.read_excel(path_datain + '\inmobiliarias_ARG_{fecha_scrap}.xlsx')
# scrap_COL = pd.read_excel(path_datain + '\inmobiliarias_COL_{fecha_scrap}.xlsx')
# scrap_ECU = pd.read_excel(path_datain + '\inmobiliarias_ECU_{fecha_scrap}.xlsx')
# scrap_PER = pd.read_excel(path_datain + '\inmobiliarias_PER_{fecha_scrap}.xlsx')

# scrap_full = pd.concat([scrap_ARG,scrap_COL,scrap_ECU,scrap_PER])